<a href="https://colab.research.google.com/github/anuragsingh17ai/Large-Language-Models--LLM-/blob/main/For_Fun_02_Finetuning_Florence_2_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# datasets: Provides a way to load and process large datasets efficiently.

# flash_attn: Provides efficient implementations of attention mechanisms.

# timm: Stands for PyTorch Image Models. A library of pre-trained computer vision models.

# einops: Provides flexible operations for tensor manipulation.
#         Used to simplify tensor reshaping, reducing the complexity of tensor operations.

# transformers: A library by Hugging Face for natural language processing.

# pillow: A Python Imaging Library (PIL) fork. Adds image processing capabilities to your Python interpreter.

# huggingface_hub: Provides utilities to interact with Hugging Face's model hub.

!pip install datasets flash_attn timm einops transformers pillow huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 22.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-n

In [2]:
import os

In [3]:
from datasets import load_dataset
# load_dataset: Function from the datasets library used to load and process datasets.
#               Facilitates the loading of datasets for training and evaluating machine learning models.

from transformers import AutoModelForCausalLM, AutoProcessor
# AutoModelForCausalLM: Class from the transformers library for loading pre-trained models for causal language modeling.
#                      Used for tasks like text generation and language modeling.
# AutoProcessor: Utility from the transformers library that combines multiple preprocessing steps
#                (e.g., tokenization, feature extraction) into a single interface.

import torch
# torch: Core library for PyTorch, an open-source deep learning framework.
#        Provides functionalities for tensor computations and building neural networks.

import os
# os: Standard Python library for interacting with the operating system.
#     Used for file and directory management, environment variables, and other OS-level operations.

from torch.utils.data import DataLoader
# DataLoader: Utility from PyTorch for loading data in batches.
#             Simplifies the process of iterating over datasets, especially for training and evaluation loops.

from tqdm.auto import tqdm
# tqdm.auto: Provides a fast, extensible progress bar for loops.
#            Used for visualizing the progress of loops and iterations, often used in training loops.

from transformers import (AdamW, AutoProcessor, get_scheduler)
from torch.utils.data import Dataset
# Dataset: Abstract class from PyTorch that represents a dataset.
#          Custom datasets should inherit from this class and override methods like __len__ and __getitem__.


In [4]:
data = load_dataset("HuggingFaceM4/DocumentVQA")  # Load the DocumentVQA dataset from the HuggingFace hub
device = "cuda" if torch.cuda.is_available() else "cpu"  # Set the device to 'cuda' if a GPU is available, otherwise use 'cpu'

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True, revision='refs/pr/6').to(device)
# Load the pre-trained model for causal language modeling and move it to the specified device
# trust_remote_code=True: Allow execution of custom code from the model repository
# revision='refs/pr/6': Use the specific revision (pull request #6) of the model repository


processor = AutoProcessor.from_pretrained("microsoft/Florence-2-base-ft", trust_remote_code=True, revision='refs/pr/6')
# Load the processor for the pre-trained model, which handles tokenization and other preprocessing steps

torch.cuda.empty_cache()  # Clear the GPU cache to free up memory


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/38 [00:00<?, ?it/s]

...

processing_florence2.py:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [13]:
def run_example(task_prompt, text_input, image):
  prompt = task_prompt + text_input  # Combine the task prompt with the text input

  if image.mode != "RGB":
    image = image.convert("RGB")  # Ensure the image is in RGB mode

  inputs = processor(text=prompt, images=image, return_tensors="pt").to(device)
  # Process the text and image inputs, and move them to the specified device

  generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=1024,
      num_beams=3
  )  # Generate text based on the inputs, with a maximum of 1024 new tokens and using beam search with 3 beams

  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
  # Decode the generated IDs into text, including special tokens

  parsed_answer = processor.post_process_generation(generated_text, task=task_prompt, image_size=(image.width, image.height))
  # Post-process the generated text to extract the final answer, considering the task and image size

  return parsed_answer  # Return the parsed answer


In [14]:
for idx in range(3):
    print(run_example("DocVQA","What do you see in the image?",data['train'][idx]['image']))


print('---'*31)
print(data)
print('----'*32)
print(data['train'][0])

{'DocVQA': '037'}
{'DocVQA': 'no'}
{'DocVQA': 'c.j cook'}
---------------------------------------------------------------------------------------------
DatasetDict({
    train: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 39463
    })
    validation: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 5349
    })
    test: Dataset({
        features: ['questionId', 'question', 'question_types', 'image', 'docId', 'ucsf_document_id', 'ucsf_document_page_no', 'answers'],
        num_rows: 5188
    })
})
--------------------------------------------------------------------------------------------------------------------------------
{'questionId': 337, 'question': 'what is the date mentioned in this letter?', 'question_types': ['handwritten', 'form'], 'image': 

In [7]:
class DoCVQADataset(Dataset):
  def __init__(self, data):
    self.data = data  # Initialize the dataset with the provided data

  def __len__(self):
    return len(self.data)  # Return the length of the dataset

  def __getitem__(self, idx):
    example = self.data[idx]  # Get the example at the specified index
    question = "" + example['question']  # Extract the question from the example
    first_answer = example['answers'][0]  # Extract the first answer from the example
    image = example['image']  # Extract the image from the example
    if image.mode != "RGB":
      image = image.convert("RGB")  # Ensure the image is in RGB mode
    return question, first_answer, image  # Return the question, first answer, and image

train_dataset = DoCVQADataset(data['train'].select(range(1000)))  # Create the training dataset with 1000 examples
val_dataset = DoCVQADataset(data['validation'].select(range(100)))  # Create the validation dataset with 100 examples


In [15]:
# Convert to Embeddings
def collate_fn(batch):
  questions, answers, images = zip(*batch)  # Unpack the batch of tuples into separate lists for questions, answers, and images
  inputs = processor(text=list(questions), images=list(images), return_tensors="pt").to(device)  # Process the lists of questions and images into tensors using the processor
  return inputs, answers   # Return the processed tensors for model input (inputs) and the list of answers

batch_size = 1  # Set the batch size for data loading
num_workers = 0  # Set the number of worker threads for data loading

In [16]:
batch_size = 1  # Set the batch size for loading data
num_workers = 0  # Set the number of worker threads to use for data loading

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=num_workers)
# Create a DataLoader for the training dataset
# 'batch_size' determines the number of samples per batch
# 'shuffle=True' enables shuffling of data at the start of each epoch
# 'collate_fn=collate_fn' specifies the custom function to combine and process samples into a batch
# 'num_workers=num_workers' sets the number of worker threads for data loading

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=num_workers)
# Create a DataLoader for the validation dataset
# 'batch_size' determines the number of samples per batch
# 'shuffle=False' disables shuffling, as validation data is typically not shuffled
# 'collate_fn=collate_fn' specifies the custom function to combine and process samples into a batch
# 'num_workers=num_workers' sets the number of worker threads for data loading


In [17]:
def train_model(train_loader, val_loader, model, processor, epochs=10, lr=1e-6):
  optimizer = AdamW(model.parameters(), lr=lr)  # Initialize the AdamW optimizer with model parameters and learning rate

  num_training_steps = epochs * len(train_loader)  # Calculate the total number of training steps (batches per epoch times number of epochs)

  lr_scheduler = get_scheduler(
      name = "linear",  # Use a linear learning rate scheduler
      optimizer = optimizer,  # Pass the optimizer to the scheduler
      num_warmup_steps = 0,  # Set the number of warmup steps to 0 (no warmup phase)
      num_training_steps = num_training_steps  # Pass the total number of training steps
  )

  for epoch in range(epochs):  # Loop over the number of epochs
    model.train()  # Set the model to training mode
    train_loss = 0  # Initialize variable to accumulate training loss
    i = -1  # Initialize a counter for batches
    for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}/{epochs}"):  # Loop over the training batches with progress bar
      i+=1  # Increment batch counter
      inputs, answers = batch  # Unpack the batch into inputs and answers
      input_ids = inputs['input_ids']  # Extract input IDs from inputs
      pixel_values = inputs['pixel_values']  # Extract pixel values from inputs
      labels = processor.tokenizer(text=answers, return_tensors="pt", padding=True, return_token_type_ids=False).input_ids.to(device)  # Tokenize answers, convert to tensors, pad, and move to device

      outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)  # Forward pass through the model with inputs and labels
      loss = outputs.loss  # Extract the loss from model outputs

      loss.backward()  # Backward pass to compute gradients
      optimizer.step()  # Update model parameters using the optimizer
      lr_scheduler.step()  # Update the learning rate based on the scheduler
      optimizer.zero_grad()  # Zero out gradients to prevent accumulation

      train_loss += loss.item()  # Accumulate training loss

    avg_train_loss = train_loss / len(train_loader)  # Calculate average training loss for the epoch
    print(f"Average Training Loss: {avg_train_loss}")  # Print average training loss

    model.eval()  # Set the model to evaluation mode
    val_loss = 0  # Initialize variable to accumulate validation loss
    with torch.inference_mode():  # Disable gradient tracking for validation
      for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}/{epochs}"):  # Loop over validation batches with progress bar
        inputs, answers = batch  # Unpack the batch into inputs and answers

        input_ids = inputs['input_ids']  # Extract input IDs from inputs
        pixel_values = inputs["pixel_values"]  # Extract pixel values from inputs
        labels = processor.tokenizer(text=answers, return_tensors="pt", padding=True, return_token_type_ids=False).input_ids.to(device)  # Tokenize answers, convert to tensors, pad, and move to device

        outputs = model(input_ids=input_ids, pixel_values=pixel_values, labels=labels)  # Forward pass through the model with inputs and labels
        loss = outputs.loss  # Extract the loss from model outputs

        val_loss += loss.item()  # Accumulate validation loss

    avg_val_loss = val_loss / len(val_loader)  # Calculate average validation loss for the epoch
    print(f"Average Validation Loss: {avg_val_loss}")  # Print average validation loss

    # Save model checkpoint
    output_dir = f"./model_checkpoints/epoch_{epoch+1}"  # Define directory path for saving model checkpoint
    os.makedirs(output_dir, exist_ok=True)  # Create the directory if it does not exist
    model.save_pretrained(output_dir)  # Save the model to the checkpoint directory
    processor.save_pretrained(output_dir)  # Save the processor to the checkpoint directory

for param in model.vision_tower.parameters():
  param.requires_grad = False  # Set all parameters in the vision tower to be non-trainable

train_model(train_loader, val_loader, model, processor, epochs=1)  # Call the train_model function with the specified parameters for 1 epoch


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Epoch 1/1:   0%|          | 0/1000 [00:00<?, ?it/s]

Average Training Loss: 2.1124536834703758


Validation Epoch 1/1:   0%|          | 0/100 [00:00<?, ?it/s]

Average Validation Loss: 1.681401052782312


In [23]:
import matplotlib.pyplot as plt
for idx in range(3):
    print(run_example("DocVQA","What is the date?",data['train'][idx]['image']))


{'DocVQA': '11/18/18'}
{'DocVQA': '11/18/18'}
{'DocVQA': 'May 9, 1995'}
